In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
import src.Utils 

In [9]:
import numpy as np
import random
import pickle
import string

import torch
from torch.utils.data import Dataset, Subset, ConcatDataset

In [12]:
ALPHABET = string.ascii_lowercase + string.digits + "."
char2idx = {c: i + 1 for i, c in enumerate(ALPHABET)}  # padding=0
idx2char = {i: c for c, i in char2idx.items()}       # Reverse mapping index -> character
vocab_size = len(char2idx) + 1

MAX_LEN = 50
def domain_to_tensor(domain):
    arr = [char2idx.get(c, 0) for c in domain.lower()][:MAX_LEN]
    arr += [0] * (MAX_LEN - len(arr))
    return torch.tensor(arr, dtype=torch.long)

def tensor_to_domain(tensor):
    """
    Converts a tensor back into a domain string.
    - Ignores padding (index=0).
    - Uses idx2char to map indices back to characters.

    Args:
    - tensor: A 1D PyTorch tensor representing the encoded domain.

    Returns:
    - str: Decoded domain name.
    """
    domain = "".join(idx2char.get(idx, "") for idx in tensor.tolist() if idx > 0)  # Ignore padding (0)
    return domain

def load_dataset(file_path):
    with open(file_path, 'rb') as file:
        dataloader = pickle.load(file)
    print(f"DataLoader loaded from {file_path}.")
    return dataloader


class DomainDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        dom, lbl = self.samples[idx]
        x = domain_to_tensor(dom)
        return x, lbl

In [13]:
benign_train_ds = load_dataset("domain2/benign_train.pkl")
dga_1_train_ds = load_dataset("domain2/dga_1_train.pkl")
dga_2_train_ds = load_dataset("domain2/dga_2_train.pkl")
dga_3_train_ds = load_dataset("domain2/dga_3_train.pkl")
dga_4_train_ds = load_dataset("domain2/dga_4_train.pkl")

    # Tạo các tập con kết hợp
all_train_datasets = [
    ConcatDataset([benign_train_ds, dga_1_train_ds]),
    ConcatDataset([benign_train_ds, dga_2_train_ds]),
    ConcatDataset([benign_train_ds, dga_3_train_ds]),
    ConcatDataset([benign_train_ds, dga_4_train_ds])
]

DataLoader loaded from domain2/benign_train.pkl.
DataLoader loaded from domain2/dga_1_train.pkl.
DataLoader loaded from domain2/dga_2_train.pkl.
DataLoader loaded from domain2/dga_3_train.pkl.
DataLoader loaded from domain2/dga_4_train.pkl.


In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=50):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)


# Transformer Model for DGA Detection
class PositionalEncodingTransformer(nn.Module):
    def __init__(self, vocab_size=vocab_size, embed_dim=64, nhead=4, num_layers=4, dim_feedforward=256, dropout=0.2,
                 num_classes=2):
        super().__init__()

        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Positional Encoding
        self.positional_encoding = PositionalEncoding(embed_dim)

        # Transformer Encoder Layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation="relu",
            layer_norm_eps=1e-5,
            batch_first=True  # Fix the warning by enabling batch_first
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Fully Connected Output Layer
        self.fc = nn.Linear(embed_dim, num_classes)

        # Dropout & Layer Normalization
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # Embedding Layer
        emb = self.embedding(x)  # (B, L, E)
        emb = self.positional_encoding(emb)  # Add positional encodings
        emb = self.dropout(emb)  # Apply dropout
        emb = self.layer_norm(emb)  # Apply Layer Normalization

        # Transformer Encoder (no permute needed now)
        out = self.transformer_encoder(emb)  # (B, L, E)

        # Extract final representation (use first token's output)
        out_final = out[:, 0, :]  # (Batch, Embed_Dim)

        # Fully Connected Output
        logits = self.fc(out_final)  # (Batch, Num_Classes)

        return logits

In [ ]:
train_set = all_train_datasets[2]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Sử dụng BCELoss cho phân loại nhị phân
criterion = nn.BCELoss()

In [15]:
def train_on_device(model, lr, momentum, trainloader, data_name, criterion, epoch = 1, clip_grad_norm=None):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    model.train()
    for _ in range(epoch):
        for (training_data, label) in tqdm(trainloader):
            if training_data.size(0) == 1:
                continue
            training_data = training_data.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            if data_name == "DOMAIN2":
                output = model(training_data)
                output = torch.softmax(output, dim=1)[:, 1]
                loss = criterion(output, label.float())
            else:
                output = model(training_data)
                loss = criterion(output, label)

            if torch.isnan(loss).any():
                src.Log.print_with_color("NaN detected in loss, stop training", "yellow")
                return False

            loss.backward()
            if clip_grad_norm and clip_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
            optimizer.step()

    return True



In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster

def Clustering_Hierarchical(interference_output, config):
    """
    Tính toán ma trận tương đồng và thực hiện phân cụm phân cấp (Hierarchical Clustering).
    
    :param interference_output: Mảng numpy chứa các vector đặc trưng đầu ra của các client.
    :param config: Dictionary chứa các tham số cấu hình, bao gồm 'threshold' để kiểm soát ngưỡng phân cụm.
    :return: Dictionary chứa các cụm, mỗi cụm là một danh sách các client_id.
    """
    
    # Lấy ngưỡng từ cấu hình
    threshold = config['threshold']
    
    # Tính toán ma trận tương đồng Cosine giữa các client
    similarity_matrix = cosine_similarity(interference_output)
    
    # Chuyển ma trận tương đồng thành khoảng cách (distance matrix)
    distance_matrix = 1 - similarity_matrix  # Cosine distance = 1 - Cosine similarity
    
    # Áp dụng thuật toán liên kết phân cấp (Hierarchical Clustering) để tính toán mối quan hệ giữa các điểm
    # Dùng phương pháp 'ward' (hoặc có thể thử các phương pháp khác như 'single', 'complete', 'average')
    Z = linkage(distance_matrix, method='ward')
    
    # Dự đoán nhóm của các client dựa trên ngưỡng (threshold)
    # fcluster dùng để tạo các nhóm (clusters) từ kết quả của linkage, ngưỡng 'threshold' xác định khi nào các nhóm được hợp nhất.
    # Nếu ngưỡng 'threshold' thấp, các nhóm sẽ được phân tách rõ ràng hơn.
    clusters = fcluster(Z, threshold, criterion='distance')
    
    # Tạo danh sách các nhóm (clusters) từ các client
    final_clusters = {}
    for idx, cluster_id in enumerate(clusters):
        if cluster_id not in final_clusters:
            final_clusters[cluster_id] = []
        final_clusters[cluster_id].append(idx)  # Lưu client id vào cluster tương ứng

    return final_clusters

# Ví dụ cấu hình và dữ liệu ngẫu nhiên
config = {
    'threshold': 0.5  # Ngưỡng phân cụm
}

# Giả lập dữ liệu interference_output với 10 client, mỗi client có 5 giá trị đầu ra
interference_output = np.random.rand(10, 5)

# Gọi hàm Clustering_Hierarchical
clusters = Clustering_Hierarchical(interference_output, config)

# In kết quả phân cụm
print("Kết quả phân cụm:", clusters)


Kết quả phân cụm: {3: [0, 4, 6], 1: [1, 2, 3, 8], 2: [5, 7], 4: [9]}


In [3]:
import numpy as np
import copy

def calculating_adjacency(interference_output): 
    """
    Tính toán ma trận tương đồng giữa các client sử dụng phép nhân vô hướng và arccos.
    
    :param interference_output: Mảng numpy chứa các vector đặc trưng đầu ra của các client.
    :return: Ma trận tương đồng giữa các client.
    """
    nclients = interference_output.shape[0]  # Lấy số lượng client (hàng của ma trận interference_output)
    sim_mat = np.zeros([nclients, nclients])  # Khởi tạo ma trận tương đồng
    
    for idx1 in range(nclients):
        for idx2 in range(nclients):
            # Lấy các vector đặc trưng của client idx1 và idx2 từ interference_output
            U1 = copy.deepcopy(interference_output[idx1])
            U2 = copy.deepcopy(interference_output[idx2])
            
            # Tính phép nhân vô hướng giữa hai vector U1 và U2
            mul = np.clip(U1.T @ U2, a_min=-1.0, a_max=1.0)  # Đảm bảo giá trị nằm trong khoảng [-1, 1]
            
            # Tính giá trị arccos của phép nhân vô hướng
            sim_mat[idx1, idx2] = 100 * np.min(np.arccos(mul))  # Chuyển đổi thành cosine similarity rồi nhân 100
            
    return sim_mat


In [4]:
# Giả sử có 5 client, mỗi client có vector đặc trưng 4 chiều
interference_output = np.random.rand(5, 4)  # Dữ liệu ngẫu nhiên cho các client

# Tính ma trận tương đồng
sim_matrix = calculating_adjacency(interference_output)

# In ma trận tương đồng
print("Ma trận tương đồng:\n", sim_matrix)


Ma trận tương đồng:
 [[ 82.57485822 124.02658646  93.58231826  77.79567072  38.99318269]
 [124.02658646 133.3529019  117.46145036 108.32099878  93.75557069]
 [ 93.58231826 117.46145036   0.           0.           0.        ]
 [ 77.79567072 108.32099878   0.           0.           0.        ]
 [ 38.99318269  93.75557069   0.           0.           0.        ]]


In [5]:
import numpy as np

# Giả sử có 5 client, mỗi client có vector đặc trưng 4 chiều
np.random.seed(42)  # Đặt seed để dữ liệu ngẫu nhiên có thể tái tạo

n_clients = 5
output_length = 4

# Tạo dữ liệu ngẫu nhiên cho U (mỗi client có một vector đặc trưng 4 chiều)
U = np.random.rand(n_clients, output_length)

# Tạo clients_idxs để chỉ định các chỉ số client
clients_idxs = list(range(n_clients))

# Kiểm tra dữ liệu ngẫu nhiên
U, clients_idxs


(array([[0.37454012, 0.95071431, 0.73199394, 0.59865848],
        [0.15601864, 0.15599452, 0.05808361, 0.86617615],
        [0.60111501, 0.70807258, 0.02058449, 0.96990985],
        [0.83244264, 0.21233911, 0.18182497, 0.18340451],
        [0.30424224, 0.52475643, 0.43194502, 0.29122914]]),
 [0, 1, 2, 3, 4])

In [ ]:
def calculating_adjacency(clients_idxs, U): 
        
    nclients = len(clients_idxs)
    
    sim_mat = np.zeros([nclients, nclients])
    for idx1 in range(nclients):
        for idx2 in range(nclients):
            #print(idx1)
            #print(U)
            #print(idx1)
            U1 = copy.deepcopy(U[clients_idxs[idx1]])
            U2 = copy.deepcopy(U[clients_idxs[idx2]])
            
            #sim_mat[idx1,idx2] = np.where(np.abs(U1.T@U2) > 1e-2)[0].shape[0]
            #sim_mat[idx1,idx2] = 10*np.linalg.norm(U1.T@U2 - np.eye(15), ord='fro')
            #sim_mat[idx1,idx2] = 100/np.pi*(np.sort(np.arccos(U1.T@U2).reshape(-1))[0:4]).sum()
            mul = np.clip(U1.T@U2 ,a_min =-1.0, a_max=1.0)
            sim_mat[idx1,idx2] = 100*np.min(np.arccos(mul))
           
    return sim_mat



[[  0.          69.53929344   0.          71.27775742   0.        ]
 [ 69.53929344  63.96396117   0.         123.19259001 115.19888867]
 [  0.           0.           0.          58.74189861  56.27191591]
 [ 71.27775742 123.19259001  58.74189861  63.55485614 105.10711102]
 [  0.         115.19888867  56.27191591 105.10711102  87.71780822]]


In [20]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster

def Clustering_Hierarchical(interference_output, config):
    """
    Tính toán ma trận tương đồng và thực hiện phân cụm phân cấp (Hierarchical Clustering).
    
    :param interference_output: Mảng numpy chứa các vector đặc trưng đầu ra của các client.
    :param config: Dictionary chứa các tham số cấu hình, bao gồm 'threshold' để kiểm soát ngưỡng phân cụm.
    :return: Số lượng cụm, nhãn các nhóm (labels), None
    """
    threshold = config['threshold']  # Lấy ngưỡng từ cấu hình
    
    # Tính toán ma trận tương đồng Cosine giữa các client
    similarity_matrix = cosine_similarity(interference_output)
    
    # Chuyển ma trận tương đồng thành khoảng cách (distance matrix)
    distance_matrix = 1 - similarity_matrix  # Cosine distance = 1 - Cosine similarity
    print(distance_matrix)
    
    # Áp dụng thuật toán phân cụm phân cấp (Hierarchical Clustering)
    Z = linkage(distance_matrix, method='ward')
    
    # Dự đoán nhóm của các client dựa trên ngưỡng (threshold)
    labels = fcluster(Z, threshold, criterion='distance')
    
    # Số lượng cụm (clusters)
    num_clusters = len(np.unique(labels))  # Đếm số lượng nhóm (cụm)
    
    return num_clusters, labels, None


# Ví dụ sử dụng

# Giả sử có 5 client với 4 chiều đặc trưng
interference_output = np.random.rand(5, 4)  # Dữ liệu ngẫu nhiên cho 5 client

# Cấu hình với ngưỡng phân cụm
config = {
    'threshold': 0.5 # Ngưỡng phân cụm
}

# Gọi hàm Clustering_Hierarchical
num_clusters, labels, _ = Clustering_Hierarchical(interference_output, config)

# In kết quả phân cụm
print("Số lượng cụm:", num_clusters)
print("Nhóm các client:", labels)


[[ 0.00000000e+00  2.04195524e-01  3.05896091e-01  1.16146762e-01
   4.32613831e-02]
 [ 2.04195524e-01  1.11022302e-16  6.34609524e-01  1.37004235e-01
   1.56800555e-01]
 [ 3.05896091e-01  6.34609524e-01  1.11022302e-16  3.15166866e-01
   4.47062583e-01]
 [ 1.16146762e-01  1.37004235e-01  3.15166866e-01 -2.22044605e-16
   2.15799280e-01]
 [ 4.32613831e-02  1.56800555e-01  4.47062583e-01  2.15799280e-01
   2.22044605e-16]]
Số lượng cụm: 2
Nhóm các client: [1 1 2 1 1]
